In [2]:
import time
import csv
from tqdm import tqdm_notebook
import re

import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
#스킨케어
skin_list = ['https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100090001&isLoginCnt=1&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100090001_MID&trackingCd=Cat100000100090001_MID&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EC%83%81%EC%84%B8_%EC%A4%91%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&t_2nd_category_type=%EC%A4%91_%EC%8B%9C%ED%8A%B8%ED%8C%A9']

In [ ]:
start=time.time() # 얼마나 걸리나 알아보기 위해 넣음
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 2)
data_list = []
for url_idx, url in enumerate(skin_list):
    df = pd.DataFrame(columns=['big_category','category','name', 'brand','price', 'sale_price', 'date', 'rate', 'id', 'skin_type', 'select_1_title', 'select_1_content', 'select_2_title', 'select_2_content', 'select_3_title', 'select_3_content', 'txt'])
    print('---------------------------------------------------------------크롤링 시작-------------------------------------------------------------------')
    driver.get(url)
    time.sleep(2)  # 페이지 로드를 기다리기 위한 기본 대기
    for row in range(2,8):
        for col in range(1,5):
            try:
                driver.execute_script(f"window.scrollBy(0, {600});")
                element = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="Contents"]/ul[{row}]/li[{col}]/div/a/img')))
                element.click()
            except Exception as e:
                print(f"row {row}, col {col}에서 요소 클릭 중 오류 발생: {e}")
            try:
                review_count=driver.find_element(By.XPATH,'//*[@id="repReview"]/em').text
                review_count = int(re.sub(",|\(|\)|건", "", review_count))
                print(review_count)
            except:
                pass
            if review_count>=100:
                big_category = driver.find_element(By.ID,"midCatNm").text
                category = driver.find_element(By.ID,'smlCatNm').text
                name = driver.find_element(By.CLASS_NAME,'prd_name').text
                brand = driver.find_element(By.ID,'moveBrandShop').text
                sale_price = driver.find_element(By.CLASS_NAME,'price-2 > strong').text
                try:
                    price = driver.find_element(By.CLASS_NAME,'price-1 > strike').text
                except:
                    price=sale_price
                driver.find_element(By.CLASS_NAME,'goods_reputation').click()
            else:
                pass
            
            scroll_script = "window.scrollTo(0, 1200);"  # 500px 위치로 스크롤
            driver.execute_script(scroll_script)
            time.sleep(1)
            review_button = driver.find_element(By.CSS_SELECTOR,f"#reviewInfo > a > span")
            review_button.click()
            time.sleep(1.2)
            # 리뷰 1페이지 수집시작
            for i in range(1,11): # 10개 댓글 데이터 크롤링 
                try:
                    date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                    rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                    id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                    skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                    select_1_title = "피부타입"
                    select_1_content = "없음"
                    select_2_title = "피부고민"
                    select_2_content = "없음"
                    select_3_title = "자극도"
                    select_3_content = "없음"
                    txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                    # DataFrame에 데이터 추가
                    data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                    time.sleep(1)
                except:
                    pass
            print(len(data_list))
            for num in range(2, 11):#2~10페이지까지 수집
                try:
                    page_num = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({num})')))
                    page_num.click()
                except Exception as e:
                    print(f"Page {num} 이동 중 예외 발생: {e}. 더 이상 페이지가 없습니다.")
                    break  # 다음 페이지가 없으면 루프 종료
                time.sleep(1.8)
                for i in range(1,11): # 10개 댓글 데이터 크롤링 
                    try:
                        date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date')))
                        date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                        rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                        id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                        skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                        select_1_title = "피부타입"
                        select_1_content = "없음"
                        select_2_title = "피부고민"
                        select_2_content = "없음"
                        select_3_title = "자극도"
                        select_3_content = "없음"
                        txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                        
                        # DataFrame에 데이터 추가
                        data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                        time.sleep(1)
                    except:
                        pass
            print(len(data_list))
            try:
                next_button = driver.find_element(By.CSS_SELECTOR,f"#gdasContentsArea > div > div.pageing > a.next")
                next_button.click()
                time.sleep(1)
            except:
                pass
            # 리뷰 11페이지 수집시작
            for i in range(1,11): # 10개 댓글 데이터 크롤링 
                try:
                    date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date')))
                    date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                    rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                    id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                    skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                    select_1_title = "피부타입"
                    select_1_content = "없음"
                    select_2_title = "피부고민"
                    select_2_content = "없음"
                    select_3_title = "자극도"
                    select_3_content = "없음"
                    txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                    # DataFrame에 데이터 추가
                    data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                    time.sleep(1)
                except:
                    break
            print(len(data_list))            
            for num in range(3, 12): #12 ~20페이지까지 수집
                try:
                    page_num = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({num})')))
                    page_num.click()
                except Exception as e:
                    print(f"Page {num} 이동 중 예외 발생: {e}. 더 이상 페이지가 없습니다.")
                    break  # 다음 페이지가 없으면 루프 종료
                time.sleep(1)
                for i in range(1,11): # 10개 댓글 데이터 크롤링 
                    try:
                        date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date')))
                        date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                        rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                        id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                        skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                        select_1_title = "피부타입"
                        select_1_content = "없음"
                        select_2_title = "피부고민"
                        select_2_content = "없음"
                        select_3_title = "자극도"
                        select_3_content = "없음"
                        txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                        
                        # DataFrame에 데이터 추가
                        data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                        time.sleep(1)
                    except:
                        break
            print(len(data_list))
            try:
                next_button = driver.find_element(By.CSS_SELECTOR,f"#gdasContentsArea > div > div.pageing > a.next")
                next_button.click()
                time.sleep(1)
            except:
                pass
            # 리뷰 21페이지 수집시작
            for i in range(1,11): # 10개 댓글 데이터 크롤링 
                try:
                    date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date')))
                    date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                    rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                    id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                    skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                    select_1_title = "피부타입"
                    select_1_content = "없음"
                    select_2_title = "피부고민"
                    select_2_content = "없음"
                    select_3_title = "자극도"
                    select_3_content = "없음"
                    txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                    # DataFrame에 데이터 추가
                    data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                    time.sleep(1)
                except:
                    break
                        
            for num in range(3, 12): #22 ~30페이지까지 수집
                try:
                    page_num = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({num})')))
                    page_num.click()
                except Exception as e:
                    print(f"Page {num} 이동 중 예외 발생: {e}. 더 이상 페이지가 없습니다.")
                    break  # 다음 페이지가 없으면 루프 종료
                time.sleep(1.8)
                for i in range(1,11): # 10개 댓글 데이터 크롤링 
                    try:
                        date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date')))
                        date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                        rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                        id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                        skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                        select_1_title = "피부타입"
                        select_1_content = "없음"
                        select_2_title = "피부고민"
                        select_2_content = "없음"
                        select_3_title = "자극도"
                        select_3_content = "없음"
                        txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                        # DataFrame에 데이터 추가
                        data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                        time.sleep(1)
                    except:
                        pass
            print(len(data_list))
            try:
                next_button = driver.find_element(By.CSS_SELECTOR,f"#gdasContentsArea > div > div.pageing > a.next")
                next_button.click()        
                time.sleep(1)
            except:
                pass
            # 리뷰 31페이지 수집시작
            
            for i in range(1,11): # 10개 댓글 데이터 크롤링 
                try:
                    date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date')))
                    date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                    rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                    id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                    skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                    select_1_title = "피부타입"
                    select_1_content = "없음"
                    select_2_title = "피부고민"
                    select_2_content = "없음"
                    select_3_title = "자극도"
                    select_3_content = "없음"
                    txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                    # DataFrame에 데이터 추가
                    data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                    time.sleep(1)
                except:
                    pass
                
            # 이전 페이지로 돌아가기
            for num in range(3, 12): #32 ~40페이지까지 수집
                try:#gdasContentsArea > div > div.pageing > a:nth-child(3)
                    page_num = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({num})')))
                    page_num.click()
                except Exception as e:
                    print(f"Page {num} 이동 중 예외 발생: {e}. 더 이상 페이지가 없습니다.")
                    break  # 다음 페이지가 없으면 루프 종료
                time.sleep(1.9)
                for i in range(1,11): # 10개 댓글 데이터 크롤링 
                    try:
                        date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date')))
                        date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                        rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text         
                        id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text                   
                        skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.tag').text
                        select_1_title = "피부타입"
                        select_1_content = "없음"
                        select_2_title = "피부고민"
                        select_2_content = "없음"
                        select_3_title = "자극도"
                        select_3_content = "없음"   
                        txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text   
                        # DataFrame에 데이터 추가
                        data_list.append([
                                big_category, category, name, brand, price, sale_price, date, rate, id,
                                skin_type, select_1_title, select_1_content, select_2_title, select_2_content,
                                select_3_title, select_3_content, txt
                            ])
                        time.sleep(1)
                    except:
                        pass
            print(len(data_list))
            # 이전 페이지로 돌아가기
            driver.back()

columns = ['big_category', 'category', 'name', 'brand', 'price', 'sale_price', 'date', 'rate', 'id',
           'skin_type', 'select_1_title', 'select_1_content', 'select_2_title', 'select_2_content',
           'select_3_title', 'select_3_content', 'txt']

df = pd.DataFrame(data_list, columns=columns)                

---------------------------------------------------------------크롤링 시작-------------------------------------------------------------------
127999
10
Page 10 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
77
77
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
77
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
77
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
77
3347
85
119
119
126
178
248
173861
258
Page 9 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
317
317
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
317
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
317
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
317
139248
327
Page 7 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
369
369
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
369
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
369
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
369
49037
375
451
451
456
508
551
48669
561
648
658
723
822
921
65272
931
985
985
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
985
Page 3 이동 중 예외 발생: Message: 
. 더 이상 페이지가 없습니다.
985
107

In [ ]:
df.to_csv("mask1.csv",encoding='utf-8-sig')